### For this assignment you will be exploring publicly available data from LendingClub.com. Lending Club connects people who need money (borrowers) with people who have money (investors). Hopefully, as an investor you would want to invest in people who showed a profile of having a high probability of paying you back. You will be creating a model that will help predict this.

### We will use lending data from 2007-2010 and be trying to classify and predict whether or not the borrower paid back their loan in full.

credit.policy: 1 if the customer meets the credit underwriting criteria of LendingClub.com, and 0 otherwise.
purpose: The purpose of the loan (takes values "credit_card", "debt_consolidation", "educational", "major_purchase", "small_business", and "all_other").
int.rate: The interest rate of the loan, as a proportion (a rate of 11% would be stored as 0.11). Borrowers judged by LendingClub.com to be more risky are assigned higher interest rates.
installment: The monthly installments owed by the borrower if the loan is funded.
log.annual.inc: The natural log of the self-reported annual income of the borrower.
dti: The debt-to-income ratio of the borrower (amount of debt divided by annual income).
fico: The FICO credit score of the borrower.
days.with.cr.line: The number of days the borrower has had a credit line.
revol.bal: The borrower's revolving balance (amount unpaid at the end of the credit card billing cycle).
revol.util: The borrower's revolving line utilization rate (the amount of the credit line used relative to total credit available).
inq.last.6mths: The borrower's number of inquiries by creditors in the last 6 months.
delinq.2yrs: The number of times the borrower had been 30+ days past due on a payment in the past 2 years.
pub.rec: The borrower's number of derogatory public records (bankruptcy filings, tax liens, or judgments).

In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import PolynomialFeatures
import warnings
warnings.filterwarnings("ignore")
np.random.seed(42)

In [47]:
df = pd.read_csv('loan_data.csv')
df.head()

,credit.policy,purpose,int.rate,installment,log.annual.inc,dti,fico,days.with.cr.line,revol.bal,revol.util,inq.last.6mths,delinq.2yrs,pub.rec,not.fully.paid
0,1,debt_consolidation,0.1189,829.10,11.350407,19.48,737,5639.958333,28854,52.1,0,0,0,0
1,1,credit_card,0.1071,228.22,11.082143,14.29,707,2760.000000,33623,76.7,0,0,0,0
2,1,debt_consolidation,0.1357,366.86,10.373491,11.63,682,4710.000000,3511,25.6,1,0,0,0
3,1,debt_consolidation,0.1008,162.34,11.350407,8.10,712,2699.958333,33667,73.2,1,0,0,0
4,1,credit_card,0.1426,102.92,11.299732,14.97,667,4066.000000,4740,39.5,0,1,0,0


In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9578 entries, 0 to 9577
Data columns (total 14 columns):
credit.policy        9578 non-null int64
purpose              9578 non-null object
int.rate             9578 non-null float64
installment          9578 non-null float64
log.annual.inc       9578 non-null float64
dti                  9578 non-null float64
fico                 9578 non-null int64
days.with.cr.line    9578 non-null float64
revol.bal            9578 non-null int64
revol.util           9578 non-null float64
inq.last.6mths       9578 non-null int64
delinq.2yrs          9578 non-null int64
pub.rec              9578 non-null int64
not.fully.paid       9578 non-null int64
dtypes: float64(6), int64(7), object(1)
memory usage: 1.0+ MB


In [49]:
y=df.pop('not.fully.paid')

In [50]:
y.value_counts()

0    8045
1    1533
Name: not.fully.paid, dtype: int64

In [51]:
df.isnull().sum()

credit.policy        0
purpose              0
int.rate             0
installment          0
log.annual.inc       0
dti                  0
fico                 0
days.with.cr.line    0
revol.bal            0
revol.util           0
inq.last.6mths       0
delinq.2yrs          0
pub.rec              0
dtype: int64

In [52]:
import pandas_profiling

In [53]:
pandas_profiling.ProfileReport(df)

Number of variables,13
Number of observations,9578
Total Missing (%),0.0%
Total size in memory,972.8 KiB
Average record size in memory,104.0 B
Numeric,11
Categorical,1
Boolean,1
Date,0
Text (Unique),0
Rejected,0


In [60]:
df['purpose'] = df['purpose'].astype('category')

In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9578 entries, 0 to 9577
Data columns (total 13 columns):
credit.policy        9578 non-null int64
purpose              9578 non-null category
int.rate             9578 non-null float64
installment          9578 non-null float64
log.annual.inc       9578 non-null float64
dti                  9578 non-null float64
fico                 9578 non-null int64
days.with.cr.line    9578 non-null float64
revol.bal            9578 non-null int64
revol.util           9578 non-null float64
inq.last.6mths       9578 non-null int64
delinq.2yrs          9578 non-null int64
pub.rec              9578 non-null int64
dtypes: category(1), float64(6), int64(6)
memory usage: 907.7 KB


In [62]:
encodedDF = pd.get_dummies(df[['purpose']])

In [63]:
encodedDF.head()


,purpose_all_other,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_major_purchase,purpose_small_business
0,0,0,1,0,0,0,0
1,0,1,0,0,0,0,0
2,0,0,1,0,0,0,0
3,0,0,1,0,0,0,0
4,0,1,0,0,0,0,0


In [64]:
df = pd.concat([encodedDF,df.drop(['credit.policy','purpose'],axis=1)],axis=1)

In [65]:
df.head(10)

,purpose_all_other,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_major_purchase,purpose_small_business,int.rate,installment,log.annual.inc,dti,fico,days.with.cr.line,revol.bal,revol.util,inq.last.6mths,delinq.2yrs,pub.rec
0,0,0,1,0,0,0,0,0.1189,829.10,11.350407,19.48,737,5639.958333,28854,52.1,0,0,0
1,0,1,0,0,0,0,0,0.1071,228.22,11.082143,14.29,707,2760.000000,33623,76.7,0,0,0
2,0,0,1,0,0,0,0,0.1357,366.86,10.373491,11.63,682,4710.000000,3511,25.6,1,0,0
3,0,0,1,0,0,0,0,0.1008,162.34,11.350407,8.10,712,2699.958333,33667,73.2,1,0,0
4,0,1,0,0,0,0,0,0.1426,102.92,11.299732,14.97,667,4066.000000,4740,39.5,0,1,0
5,0,1,0,0,0,0,0,0.0788,125.13,11.904968,16.98,727,6120.041667,50807,51.0,0,0,0
6,0,0,1,0,0,0,0,0.1496,194.02,10.714418,4.00,667,3180.041667,3839,76.8,0,0,1
7,1,0,0,0,0,0,0,0.1114,131.22,11.002100,11.08,722,5116.000000,24220,68.6,0,0,0
8,0,0,0,0,1,0,0,0.1134,87.19,11.407565,17.25,682,3989.000000,69909,51.1,1,0,0
9,0,0,1,0,0,0,0,0.1221,84.12,10.203592,10.00,707,2730.041667,5630,23.0,1,0,0


In [66]:
train, test, y_train, y_test = train_test_split(df,y,test_size = 0.2,random_state=101)

In [67]:
lr = LogisticRegression()
lr.fit(train, y_train)
y_pred = lr.predict(test)
print('Accuracy score baseline:', accuracy_score(y_test,y_pred))

Accuracy score baseline: 0.8465553235908142


In [71]:
def fit_predict(train, test, y_train, y_test, scaler, max_depth,
               criterion = 'entropy', max_features = 1, min_samples_split = 4):
    train_scaled = scaler.fit_transform(train)
    test_scaled = scaler.transform(test)
    dt = DecisionTreeClassifier(criterion, max_depth=max_depth,
                               random_state=42, max_features = max_features,
                               min_samples_split = min_samples_split)
    dt.fit(train_scaled, y_train)
    y_pred = dt.predict(test_scaled)
    print(accuracy_score(y_test, y_pred))

In [72]:
dt = DecisionTreeClassifier()
dt.fit(train, y_train)
y_pred = dt.predict(test)
print(accuracy_score(y_test,y_pred))

0.7437369519832986


### max depth tunning

In [75]:
for i in range(1,20):
    print('Accuracy score using max_depth =', i, end= ': ')
    fit_predict(train, test, y_train, y_test, StandardScaler(), i)

Accuracy score using max_depth = 1: 0.8470772442588727
Accuracy score using max_depth = 2: 0.8470772442588727
Accuracy score using max_depth = 3: 0.8465553235908142
Accuracy score using max_depth = 4: 0.8470772442588727
Accuracy score using max_depth = 5: 0.8465553235908142
Accuracy score using max_depth = 6: 0.8465553235908142
Accuracy score using max_depth = 7: 0.843945720250522
Accuracy score using max_depth = 8: 0.8397703549060542
Accuracy score using max_depth = 9: 0.8423799582463466
Accuracy score using max_depth = 10: 0.8366388308977035
Accuracy score using max_depth = 11: 0.8324634655532359
Accuracy score using max_depth = 12: 0.8303757828810021
Accuracy score using max_depth = 13: 0.8282881002087683
Accuracy score using max_depth = 14: 0.8204592901878914
Accuracy score using max_depth = 15: 0.819937369519833
Accuracy score using max_depth = 16: 0.8094989561586639
Accuracy score using max_depth = 17: 0.8021920668058455
Accuracy score using max_depth = 18: 0.7990605427974948
Acc

### Max feature tunning

In [76]:
for i in np.arange(0.1, 1.0, 0.1):
    print('Accuracy score using max_features =', i, end= ': ')
    fit_predict(train, test, y_train, y_test, StandardScaler(), max_depth = 4, max_features=i)

Accuracy score using max_features = 0.1: 0.8470772442588727
Accuracy score using max_features = 0.2: 0.8470772442588727
Accuracy score using max_features = 0.30000000000000004: 0.8470772442588727
Accuracy score using max_features = 0.4: 0.8444676409185804
Accuracy score using max_features = 0.5: 0.8434237995824635
Accuracy score using max_features = 0.6: 0.8455114822546973
Accuracy score using max_features = 0.7000000000000001: 0.8444676409185804
Accuracy score using max_features = 0.8: 0.842901878914405
Accuracy score using max_features = 0.9: 0.843945720250522


### Min Sample split tunning

In [78]:
for i in range(2,10):
    print('Accuracy score using min sample split=', i, end= ': ')
    fit_predict(train, test, y_train, y_test, StandardScaler(), 4, max_features= 0.1, min_samples_split=i)

Accuracy score using min sample split= 2: 0.8470772442588727
Accuracy score using min sample split= 3: 0.8470772442588727
Accuracy score using min sample split= 4: 0.8470772442588727
Accuracy score using min sample split= 5: 0.8470772442588727
Accuracy score using min sample split= 6: 0.8470772442588727
Accuracy score using min sample split= 7: 0.8470772442588727
Accuracy score using min sample split= 8: 0.8470772442588727
Accuracy score using min sample split= 9: 0.8470772442588727


### Criterion Tunning

In [79]:
for i in ['gini', 'entropy']:
    print('Accuracy score using criterion tunning=', i, end= ': ')
    fit_predict(train, test, y_train, y_test, StandardScaler(), 4, 
                max_features= 0.1, min_samples_split=2,criterion = i)

Accuracy score using criterion tunning= gini: 0.8465553235908142
Accuracy score using criterion tunning= entropy: 0.8470772442588727


In [80]:
def create_poly(train, test, degree):
    poly = PolynomialFeatures(degree=degree)
    train_poly = poly.fit_transform(train)
    test_poly = poly.fit_transform(test)
    return train_poly, test_poly

In [81]:
for degree in [1,2,3,4]:
    train_poly, test_poly = create_poly(train, test, degree)
    print('Polynomial degree',degree)
    fit_predict(train_poly, test_poly, y_train, y_test, StandardScaler(), 4,
               max_features = 0.1, min_samples_split = 2, criterion = 'entropy')
    print(10*'-')
    
train_poly, test_poly = create_poly(train, test, 2)

Polynomial degree 1
0.8470772442588727
----------
Polynomial degree 2
0.8434237995824635
----------
Polynomial degree 3
0.8455114822546973
----------
Polynomial degree 4
0.8470772442588727
----------


### Random Forest

In [83]:
from sklearn.ensemble import RandomForestClassifier

In [84]:
rf = RandomForestClassifier(criterion='gini')

In [85]:
rf.fit(train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [86]:
pred_rf = rf.predict(test)

In [87]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,pred_rf))

0.8382045929018789


In [88]:
from sklearn.model_selection import GridSearchCV

In [89]:
params = {'n_estimators':[200,500,700],'max_depth':[10,15,18,20],
         'min_samples_leaf':[3,5,7]}

In [90]:
gs = GridSearchCV(rf,params,verbose=3)

In [91]:
gs.fit(train,y_train)

Fitting 3 folds for each of 36 candidates, totalling 108 fits
[CV] max_depth=10, min_samples_leaf=3, n_estimators=200 ..............
[CV]  max_depth=10, min_samples_leaf=3, n_estimators=200, score=0.8367906066536204, total=   1.9s
[CV] max_depth=10, min_samples_leaf=3, n_estimators=200 ..............


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.1s remaining:    0.0s


[CV]  max_depth=10, min_samples_leaf=3, n_estimators=200, score=0.8386844166014096, total=   2.0s
[CV] max_depth=10, min_samples_leaf=3, n_estimators=200 ..............


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    4.4s remaining:    0.0s


[CV]  max_depth=10, min_samples_leaf=3, n_estimators=200, score=0.8378378378378378, total=   2.1s
[CV] max_depth=10, min_samples_leaf=3, n_estimators=500 ..............
[CV]  max_depth=10, min_samples_leaf=3, n_estimators=500, score=0.8383561643835616, total=   5.2s
[CV] max_depth=10, min_samples_leaf=3, n_estimators=500 ..............
[CV]  max_depth=10, min_samples_leaf=3, n_estimators=500, score=0.8386844166014096, total=   5.2s
[CV] max_depth=10, min_samples_leaf=3, n_estimators=500 ..............
[CV]  max_depth=10, min_samples_leaf=3, n_estimators=500, score=0.8382295338817078, total=   4.7s
[CV] max_depth=10, min_samples_leaf=3, n_estimators=700 ..............
[CV]  max_depth=10, min_samples_leaf=3, n_estimators=700, score=0.837573385518591, total=   7.0s
[CV] max_depth=10, min_samples_leaf=3, n_estimators=700 ..............
[CV]  max_depth=10, min_samples_leaf=3, n_estimators=700, score=0.8386844166014096, total=   6.9s
[CV] max_depth=10, min_samples_leaf=3, n_estimators=700 ..

[CV]  max_depth=15, min_samples_leaf=7, n_estimators=700, score=0.8383561643835616, total=   8.0s
[CV] max_depth=15, min_samples_leaf=7, n_estimators=700 ..............
[CV]  max_depth=15, min_samples_leaf=7, n_estimators=700, score=0.8382928739232577, total=   7.8s
[CV] max_depth=15, min_samples_leaf=7, n_estimators=700 ..............
[CV]  max_depth=15, min_samples_leaf=7, n_estimators=700, score=0.8382295338817078, total=   8.3s
[CV] max_depth=18, min_samples_leaf=3, n_estimators=200 ..............
[CV]  max_depth=18, min_samples_leaf=3, n_estimators=200, score=0.8379647749510764, total=   2.6s
[CV] max_depth=18, min_samples_leaf=3, n_estimators=200 ..............
[CV]  max_depth=18, min_samples_leaf=3, n_estimators=200, score=0.8386844166014096, total=   2.5s
[CV] max_depth=18, min_samples_leaf=3, n_estimators=200 ..............
[CV]  max_depth=18, min_samples_leaf=3, n_estimators=200, score=0.836662749706228, total=   2.6s
[CV] max_depth=18, min_samples_leaf=3, n_estimators=500 ..

[CV]  max_depth=20, min_samples_leaf=7, n_estimators=200, score=0.8390759592795615, total=   2.2s
[CV] max_depth=20, min_samples_leaf=7, n_estimators=200 ..............
[CV]  max_depth=20, min_samples_leaf=7, n_estimators=200, score=0.8386212299255777, total=   2.2s
[CV] max_depth=20, min_samples_leaf=7, n_estimators=500 ..............
[CV]  max_depth=20, min_samples_leaf=7, n_estimators=500, score=0.8383561643835616, total=   5.5s
[CV] max_depth=20, min_samples_leaf=7, n_estimators=500 ..............
[CV]  max_depth=20, min_samples_leaf=7, n_estimators=500, score=0.8382928739232577, total=   5.8s
[CV] max_depth=20, min_samples_leaf=7, n_estimators=500 ..............
[CV]  max_depth=20, min_samples_leaf=7, n_estimators=500, score=0.8378378378378378, total=   5.9s
[CV] max_depth=20, min_samples_leaf=7, n_estimators=700 ..............
[CV]  max_depth=20, min_samples_leaf=7, n_estimators=700, score=0.837573385518591, total=   8.1s
[CV] max_depth=20, min_samples_leaf=7, n_estimators=700 ..

[Parallel(n_jobs=1)]: Done 108 out of 108 | elapsed: 10.9min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [200, 500, 700], 'max_depth': [10, 15, 18, 20], 'min_samples_leaf': [3, 5, 7]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [92]:
gs.best_params_

{'max_depth': 20, 'min_samples_leaf': 3, 'n_estimators': 700}

In [93]:
gs.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=3, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=700, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [95]:
rf1 = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                           max_depth=20, max_features='auto', max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=3, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=700, n_jobs=1,
                           oob_score=True, random_state=101, verbose=0,
                           warm_start=False)

In [96]:
rf1.fit(train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=3, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=700, n_jobs=1,
            oob_score=True, random_state=101, verbose=0, warm_start=False)

In [97]:
rf1.oob_score_

0.8371182458888019

In [98]:
pred_rf1 = rf1.predict(test)

In [99]:
print(accuracy_score(y_test,pred_rf1))

0.8470772442588727


In [100]:
rf1.feature_importances_

array([0.01108376, 0.00579265, 0.01263026, 0.00330451, 0.00522013,
       0.00209029, 0.01252235, 0.11033317, 0.11604686, 0.10804861,
       0.10353267, 0.0852303 , 0.11193036, 0.11239541, 0.11199535,
       0.07135686, 0.01029957, 0.00618689])

In [101]:
sorted(list(zip(rf1.feature_importances_,train.columns)),reverse=True)

[(0.11604686235100856, 'installment'),
 (0.11239541419408874, 'revol.bal'),
 (0.11199535271947572, 'revol.util'),
 (0.11193036163345264, 'days.with.cr.line'),
 (0.11033316904054125, 'int.rate'),
 (0.10804860511115291, 'log.annual.inc'),
 (0.10353266660616904, 'dti'),
 (0.08523030305497716, 'fico'),
 (0.07135685827701271, 'inq.last.6mths'),
 (0.012630257954099463, 'purpose_debt_consolidation'),
 (0.012522352377548973, 'purpose_small_business'),
 (0.011083759436649096, 'purpose_all_other'),
 (0.010299566417078918, 'delinq.2yrs'),
 (0.006186889018194423, 'pub.rec'),
 (0.00579264721918362, 'purpose_credit_card'),
 (0.005220129735632066, 'purpose_home_improvement'),
 (0.003304513982240676, 'purpose_educational'),
 (0.0020902908714939047, 'purpose_major_purchase')]

In [102]:
df.corr()

,purpose_all_other,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_major_purchase,purpose_small_business,int.rate,installment,log.annual.inc,dti,fico,days.with.cr.line,revol.bal,revol.util,inq.last.6mths,delinq.2yrs,pub.rec
purpose_all_other,1.000000,-0.220935,-0.475848,-0.109300,-0.150359,-0.124004,-0.149076,-0.124000,-0.203103,-0.080077,-0.125825,0.067184,-0.056574,-0.067728,-0.138535,0.017795,0.016658,-0.030451
purpose_credit_card,-0.220935,1.000000,-0.326850,-0.075076,-0.103279,-0.085176,-0.102397,-0.042109,0.000774,0.072942,0.084476,-0.012512,0.046220,0.072316,0.091321,-0.033640,-0.008817,0.014842
purpose_debt_consolidation,-0.475848,-0.326850,1.000000,-0.161698,-0.222441,-0.183451,-0.220542,0.123607,0.161658,-0.026214,0.179149,-0.154132,-0.009318,0.005785,0.211869,-0.044240,-0.000697,0.026845
purpose_educational,-0.109300,-0.075076,-0.161698,1.000000,-0.051094,-0.042138,-0.050658,-0.019618,-0.094510,-0.119799,-0.035325,-0.013012,-0.042621,-0.034743,-0.053128,0.024243,-0.002214,-0.013521
purpose_home_improvement,-0.150359,-0.103279,-0.222441,-0.051094,1.000000,-0.057967,-0.069687,-0.050697,0.023024,0.116375,-0.092788,0.097474,0.068087,0.003258,-0.114449,0.043827,-0.013098,0.004704
purpose_major_purchase,-0.124004,-0.085176,-0.183451,-0.042138,-0.057967,1.000000,-0.057472,-0.068978,-0.079836,-0.031020,-0.077719,0.067129,-0.020561,-0.062395,-0.108079,-0.001445,0.004085,-0.011734
purpose_small_business,-0.149076,-0.102397,-0.220542,-0.050658,-0.069687,-0.057472,1.000000,0.151247,0.145654,0.091540,-0.069245,0.063292,0.034883,0.083069,-0.060962,0.042567,-0.004148,-0.005595
int.rate,-0.124000,-0.042109,0.123607,-0.019618,-0.050697,-0.068978,0.151247,1.000000,0.276140,0.056383,0.220006,-0.714821,-0.124022,0.092527,0.464837,0.202780,0.156079,0.098162
installment,-0.203103,0.000774,0.161658,-0.094510,0.023024,-0.079836,0.145654,0.276140,1.000000,0.448102,0.050202,0.086039,0.183297,0.233625,0.081356,-0.010419,-0.004368,-0.032760
log.annual.inc,-0.080077,0.072942,-0.026214,-0.119799,0.116375,-0.031020,0.091540,0.056383,0.448102,1.000000,-0.054065,0.114576,0.336896,0.372140,0.054881,0.029171,0.029203,0.016506


### LOGISTIC REGRESSION ACCURACY BASELINE -> 0.8465553235908142
### DECISION TREE ACCURACY BASELINE -> 0.7437369519832986
### DECISION TREE ACCURACY AFTER TUNNING -> 0.8470772442588727
### RANDOM FOREST ACCURACY BASELINE -> 0.8382045929018789
### RANDOM FOREST GRIDSEARRCH CV -> 0.8470772442588727